### **Topic**
Movie Recommendation System using content-based filtering

**Recommender System** is a system that seeks to predict or filter preferences according to the user's choices. Recommender systems are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general.

Recommender systems produce a list of recommendations in any of the two ways -

**Collaborative filtering** approaches build a model from the user's past behaviour (i.e. items purchased or searched by the user) as well as similar decisions made by other users. This model is then used to predict items (or ratings for items) that users may have an interest in.

**Content-based filtering** approaches use a series of discrete characteristics of an item in order to recommend additional items with similar properties.Content-based filtering methods are totally based on a description of the item and a profile of the user's preferences. It recommends items based on the user's past preferences.

### **Import Libaries**

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### **Import DataSet**

In [2]:
df = pd.read_csv(r'https://raw.githubusercontent.com/YBI-Foundation/Dataset/main/Movies%20Recommendation.csv')
df.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [3]:
df.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

### Rename columns for shorter headers

In [4]:
df.rename(columns=lambda a: a.removeprefix("Movie_").lower(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  4760 non-null   int64  
 1   title               4760 non-null   object 
 2   genre               4760 non-null   object 
 3   language            4760 non-null   object 
 4   budget              4760 non-null   int64  
 5   popularity          4760 non-null   float64
 6   release_date        4760 non-null   object 
 7   revenue             4760 non-null   int64  
 8   runtime             4758 non-null   float64
 9   vote                4760 non-null   float64
 10  vote_count          4760 non-null   int64  
 11  homepage            1699 non-null   object 
 12  keywords            4373 non-null   object 
 13  overview            4757 non-null   object 
 14  production_house    4760 non-null   object 
 15  production_country  4760 non-null   object 
 16  spoken

In [5]:
df.shape

(4760, 21)

In [6]:
df.describe()

,id,budget,popularity,revenue,runtime,vote,vote_count
count,4760.000000,4.760000e+03,4760.000000,4.760000e+03,4758.000000,4760.000000,4760.000000
mean,2382.566387,2.920129e+07,21.599510,8.263743e+07,107.184111,6.113866,692.508403
std,1377.270159,4.075620e+07,31.887919,1.630554e+08,21.960332,1.141294,1235.007337
min,1.000000,0.000000e+00,0.000372,0.000000e+00,0.000000,0.000000,0.000000
25%,1190.750000,9.257500e+05,4.807074,0.000000e+00,94.000000,5.600000,55.000000
50%,2380.500000,1.500000e+07,13.119058,1.944716e+07,104.000000,6.200000,238.000000
75%,3572.250000,4.000000e+07,28.411929,9.341276e+07,118.000000,6.800000,740.250000
max,4788.000000,3.800000e+08,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


### **Get feature selection**

In [7]:
selected_features = df[['genre','keywords','tagline','cast','director']].fillna('')

In [8]:
selected_features.shape

(4760, 5)

In [9]:
selected_features.head()

,genre,keywords,tagline,cast,director
0,Crime Comedy,hotel new year's eve witch bet hotel room,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,Adventure Action Science Fiction,android galaxy hermit death star lightsaber,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,Animation Family,father son relationship harbor underwater fish...,"There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,Comedy Drama Romance,vietnam veteran hippie mentally disabled runni...,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,Drama,male nudity female nudity adultery midlife cri...,Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes


In [10]:
combined_features = selected_features['genre']+' '+selected_features['keywords']+' '+selected_features['tagline']+' '+selected_features['cast']+' '+selected_features['director']
combined_features.shape

(4760,)

In [11]:
combined_features

0       Crime Comedy hotel new year's eve witch bet ho...
1       Adventure Action Science Fiction android galax...
2       Animation Family father son relationship harbo...
3       Comedy Drama Romance vietnam veteran hippie me...
4       Drama male nudity female nudity adultery midli...
                              ...                        
4755    Horror  The hot spot where Satan's waitin'. Li...
4756    Comedy Family Drama  It’s better to stand out ...
4757    Thriller Drama christian film sex trafficking ...
4758                                           Family    
4759    Documentary music actors legendary perfomer cl...
Length: 4760, dtype: object

### **Vectorization of features**

TfidfVectorizer will create a matrix where each row represents a movie and each column represents a feature (i.e., a keyword). The value in each cell of the matrix will be the weight of the corresponding feature in the corresponding movie.

In [12]:
tfidf = TfidfVectorizer()

In [13]:
feature_vectors = tfidf.fit_transform(combined_features)
feature_vectors.shape

(4760, 17258)

In [14]:
print(feature_vectors)

  (0, 617)	0.1633382144407513
  (0, 492)	0.1432591540388685
  (0, 15413)	0.1465525095337543
  (0, 9675)	0.14226057295252661
  (0, 9465)	0.1659841367820977
  (0, 1390)	0.16898383612799558
  (0, 7825)	0.09799561597509843
  (0, 1214)	0.13865857545144072
  (0, 729)	0.13415063359531618
  (0, 13093)	0.1432591540388685
  (0, 15355)	0.10477815972666779
  (0, 9048)	0.0866842116160778
  (0, 11161)	0.06250380151644369
  (0, 16773)	0.17654247479915475
  (0, 5612)	0.08603537588547631
  (0, 16735)	0.10690083751525419
  (0, 7904)	0.13348000542112332
  (0, 15219)	0.09800472886453934
  (0, 11242)	0.07277788238484746
  (0, 3878)	0.11998399582562203
  (0, 5499)	0.11454057510303811
  (0, 7071)	0.19822417598406614
  (0, 7454)	0.14745635785412262
  (0, 1495)	0.19712637387361423
  (0, 9206)	0.15186283580984414
  :	:
  (4757, 5455)	0.12491480594769522
  (4757, 2967)	0.16273475835631626
  (4757, 8464)	0.23522565554066333
  (4757, 6938)	0.17088173678136628
  (4757, 8379)	0.17480603856721913
  (4757, 15303)	0.07

### **Get similarity score using Cosine Similarity**

The cosine similarity metric is used to compute the similarity between each pair of movies.

The result will be a similarity matrix where each row and column represents a movie, and each cell represents the similarity score between the corresponding pair of movies.

In [15]:
similarity = cosine_similarity(feature_vectors)
similarity.shape

(4760, 4760)

In [16]:
similarity

array([[1.        , 0.01351235, 0.03570468, ..., 0.        , 0.        ,
        0.        ],
       [0.01351235, 1.        , 0.00806674, ..., 0.        , 0.        ,
        0.        ],
       [0.03570468, 0.00806674, 1.        , ..., 0.        , 0.08014876,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08014876, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

### **Get movie name from user and validate for closest spelling**

In [17]:
movie_name = input("Enter your favourite movie name: ")

Enter your favourite movie name:  The Fault in Our Stars


In [18]:
list_of_all_titles = df['title'].tolist()

In [19]:
close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
close_matches

['The Fault in Our Stars', 'The Last Sin Eater', 'The Cat in the Hat']

In [20]:
closest_match = close_matches[0]
closest_match

'The Fault in Our Stars'

In [21]:
id_of_matched_movie = df[df.title == closest_match]['id'].values[0]
id_of_matched_movie

4332

In [22]:
similarity_score = list(enumerate(similarity[id_of_matched_movie]))
len(similarity_score)

4760

In [23]:
similarity_score[:5]

[(0, 0.0485769920173819),
 (1, 0.0),
 (2, 0.008656430609540355),
 (3, 0.030391543037549307),
 (4, 0.0)]

### **Get all movies sorted based on the similarity score wrt the fav movie**

In [24]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similar_movies[:5]

[(4332, 0.9999999999999997),
 (4648, 0.27790266231185523),
 (4007, 0.17849466494575128),
 (1302, 0.1735350077784387),
 (1895, 0.15321795283833556)]

In [25]:
print('Top 30 movies Sugested for You :\n')
for i, movie in enumerate(sorted_similar_movies[:30]):
    id = movie[0]
    title_from_id = df[df.id == id]['title'].values[0]
    print(i+1,'.',title_from_id)

Top 30 movies Sugested for You :

1 . The Fault in Our Stars
2 . The Purge: Election Year
3 . Side Effects
4 . The Edge
5 . Touching the Void
6 . Safe Haven
7 . Fun Size
8 . PCU
9 . Troop Beverly Hills
10 . Ghost World
11 . Oscar and Lucinda
12 . Cleopatra
13 . The Punisher
14 . Nightcrawler
15 . Abduction
16 . The Namesake
17 . Alive
18 . Last Holiday
19 . The Ballad of Jack and Rose
20 . Kiss the Girls
21 . The Five-Year Engagement
22 . The Last Five Years
23 . Zombie Hunter
24 . Ninja Assassin
25 . Head Over Heels
26 . Inherent Vice
27 . Chasing Liberty
28 . Ghost Town
29 . Monster-in-Law
30 . Sympathy for Lady Vengeance


### **Top 10 movie recommendation system**

In [26]:
movie_name = input('Enter your favourite movie name: ')
list_of_all_titles = df['title'].tolist()
close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
closest_match = close_matches[0]
id_of_movie = df[df.title == closest_match]['id'].values[0]
similarity_score = list(enumerate(similarity[id_of_movie]))
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Top 10 movies suggested for you:\n')
for i, movie in enumerate(sorted_similar_movies[:10]):
    id = movie[0]
    title_from_id = df[df.id == id]['title'].values[0]
    print(i+1,'.',title_from_id)

Enter your favourite movie name:  No Escape


Top 10 movies suggested for you:

1 . No Escape
2 . The Damned United
3 . Little Boy
4 . The Beast from 20,000 Fathoms
5 . Insidious
6 . Your Sister's Sister
7 . The Inhabited Island
8 . On Deadly Ground
9 . Tank Girl
10 . Sea of Love
